<h2 style="color:blue" align="center">Convolutional Neural Network (CNN) on CIFAR</h2>

#### Import the necessary libraries

In [1]:
import tensorflow as tf 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import random as python_random
import time, math 

from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D 
from keras.callbacks import Callback, CSVLogger
from tensorflow.keras.utils import to_categorical 

import warnings
warnings.filterwarnings('ignore')

2022-10-15 18:24:00.366439: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 18:24:02.026804: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


#### Reserve memory for the execution

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


2022-10-15 18:24:26.149282: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 18:24:31.406254: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1621 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


#### Load and split the dataset into training and testing set

CIFAR 10

In [3]:
(X_train,y_train),(X_test,y_test) = keras.datasets.cifar10.load_data()

CIFAR 100

In [4]:
#(X_train,y_train),(X_test,y_test) = keras.datasets.cifar100.load_data()

Here we see there are 50000 training images and 1000 test images

#### Normalize the input 

In [5]:
X_train_normalized = X_train / 255.0
X_test_normalized = X_test / 255.0

#### One hot encoding

In [6]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#### Define DNN model

In [7]:
python_random.seed(7)
np.random.seed(7)
tf.random.set_seed(7)
opti_name = ''

def get_model(): 
    model = Sequential([
        # CNN layers
        Conv2D(filters=32, kernel_size=(3,3), activation="relu", input_shape=(32,32,3)),
        MaxPooling2D((2,2)),        
        Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
        MaxPooling2D((2,2)),
        
        # Dense layer
        Flatten(),
        Dense(128, activation = 'relu'),  
        # For CIFAR-10
        Dense(10, activation = 'softmax')
        # For CIFAR-100
        #Dense(100, activation = 'softmax')
    ])
    
    global opti_name
    
    # General SGD
    #opti = keras.optimizers.SGD(learning_rate=0.001)
    #opti_name = 'SGD'
    
    # SGD with momentum
    #opti = keras.optimizers.SGD(learning_rate=0.001, momentum=0.4)
    #opti_name = 'SGD with momentum'
    
    # SGD with Nesterov momentum 
    #opti = keras.optimizers.SGD(learning_rate=0.001, momentum=0.4, nesterov=True)
    #opti_name = 'SGD with Nesterov momentum'
    
    # RMSprop 
    #opti = keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.4)
    #opti_name = 'RMSprop'
    
    # Adam
    opti = keras.optimizers.Adam(learning_rate=0.001) 
    opti_name = 'Adam'
    
    # Adamax
    #opti = keras.optimizers.Adamax(learning_rate=0.001) 
    #opti_name = 'Adamax' 
    
    # Nadam
    #opti = keras.optimizers.Nadam(learning_rate=0.001) 
    #opti_name = 'Nadam'  
    
    model.compile(
        optimizer = opti,
        loss = 'categorical_crossentropy',
        metrics = ['accuracy']
    )
    
    return model

#### Custom callbacks

For generic optimizer model

In [8]:
# Get the best of base-line model and set it as stopping criteria in HM-based model
generic_best = 0

class CustomCallbackGeneric(Callback):  
    def on_epoch_end(self, epoch, logs={}): 
        global generic_best
        accuracy = round(logs.get('accuracy'), 4) 
        
        if epoch == 0:
            generic_best = accuracy     
        
        if epoch > 0 and accuracy < generic_best :
            generic_best = accuracy

For HM based optimizer model

In [9]:
class CustomCallbackHM(Callback):  
    initial_weights = 0
    previous_weights = 0
    call_hm = 0    
     
    def on_train_begin(self, logs=None):
        self.initial_weights = model_hm.get_weights() 
        self.initial_weights = np.array(self.initial_weights,dtype=object)
        self.previous_weights = self.initial_weights
        # Harmonic mean based weights calculation
        self.call_hm = np.vectorize(self.apply_hm)  

    def on_epoch_end(self, epoch, logs={}): 
         # Set the stopping criteria at (stop_at) the MAE obtained from the baseline model 
        global generic_best 
        
        num_layers = len(model_hm.layers)  
        current_weights = model_hm.get_weights()
        current_weights = np.array(current_weights,dtype=object)        

        for i in range(len(current_weights)):  
            # Harmonic mean based weights calculation
            tensor1 = tf.convert_to_tensor(self.previous_weights[i])
            tensor2 = tf.convert_to_tensor(current_weights[i])
            current_weights[i] = self.call_hm(tensor1, tensor2, epoch)   
            
        # Updating the model with new weights
        model_hm.set_weights(current_weights.tolist())
        self.previous_weights = current_weights 
        
        #Stopping criteria
        #if(round(logs.get('accuracy'), 4) < generic_best):  
            #self.model.stop_training = True
        
    def apply_hm(self, v1, v2, epoch):  
        r = 0
        if epoch < 50:
            r = 1
        elif epoch > 50 and epoch < 100:
            r = 0.75
        elif epoch > 100 and epoch < 150:
            r = 0.5
        else:  
            r = 0.25 
            
        if v1==0 or v2==0:
            return v2
        elif v1 > v2:
            t1 = abs(v1)
            t2 = abs(v2)
            hm = 2*t1*t2/(t1+t2)
            min1 = min(t1,t2)
            diff = abs(hm-min1) * r 
            v2 = v2 - diff
            return v2  
        elif v1 < v2:
            t1 = abs(v1)
            t2 = abs(v2)
            hm = 2*t1*t2/(t1+t2)
            min1 = min(t1,t2)
            diff = abs(hm-min1) * r 
            v2 = v2 + diff
            return v2   
        else:
            return v2   

#### Training

Generic opimizer model

In [10]:
model = get_model() 
weights = model.get_weights() 
num_epochs = 100
batch_size = 256

To record loss and accuracy in CSV file

In [11]:
generic_file = '5.CIFAR (CNN)'+opti_name+' optimizer.csv'
hm_file = '5.CIFAR (CNN)'+opti_name+' HM-based optimizer.csv' 
logger_generic_model = CSVLogger(generic_file, append = False, separator=',')
logger_hm_model = CSVLogger(hm_file, append = False, separator=',')

Generic optimizer model

In [12]:
model_wihtout_hm = get_model()
model_wihtout_hm.set_weights(weights) 
st = time.time() 
model_wihtout_hm.fit(X_train_normalized, y_train, epochs = num_epochs, verbose=1, callbacks=[CustomCallbackGeneric(), logger_generic_model], batch_size=batch_size) 
et = time.time()
elapsed_training_time_generic = round(et - st, 4)
print('Execution time:', elapsed_training_time_generic, 'seconds') 

Epoch 1/100


2022-10-15 18:24:55.074221: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2022-10-15 18:24:57.996666: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-15 18:24:58.012786: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-15 18:24:58.012893: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-10-15 18:24:58.014371: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-15 18:24:58.014629: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-10-15 18:24:58.196931: I tensorflow/stream_executor/cuda/c

196/196 [==============================] - 7s 4ms/step - loss: 1.6398 - accuracy: 0.4136
Epoch 2/100
196/196 [==============================] - 1s 3ms/step - loss: 1.2965 - accuracy: 0.5405
Epoch 3/100
196/196 [==============================] - 1s 3ms/step - loss: 1.1709 - accuracy: 0.5884
Epoch 4/100
196/196 [==============================] - 1s 3ms/step - loss: 1.0805 - accuracy: 0.6236
Epoch 5/100
196/196 [==============================] - 1s 3ms/step - loss: 1.0143 - accuracy: 0.6474
Epoch 6/100
196/196 [==============================] - 1s 3ms/step - loss: 0.9545 - accuracy: 0.6688
Epoch 7/100
196/196 [==============================] - 1s 3ms/step - loss: 0.9030 - accuracy: 0.6878
Epoch 8/100
196/196 [==============================] - 1s 3ms/step - loss: 0.8569 - accuracy: 0.7050
Epoch 9/100
196/196 [==============================] - 1s 3ms/step - loss: 0.8101 - accuracy: 0.7200
Epoch 10/100
196/196 [==============================] - 1s 3ms/step - loss: 0.7812 - accuracy: 0.7284
E

196/196 [==============================] - 1s 3ms/step - loss: 6.8036e-04 - accuracy: 1.0000
Epoch 82/100
196/196 [==============================] - 1s 3ms/step - loss: 6.2767e-04 - accuracy: 1.0000
Epoch 83/100
196/196 [==============================] - 1s 3ms/step - loss: 6.0648e-04 - accuracy: 1.0000
Epoch 84/100
196/196 [==============================] - 1s 3ms/step - loss: 5.3777e-04 - accuracy: 1.0000
Epoch 85/100
196/196 [==============================] - 1s 3ms/step - loss: 4.9784e-04 - accuracy: 1.0000
Epoch 86/100
196/196 [==============================] - 1s 3ms/step - loss: 4.6206e-04 - accuracy: 1.0000
Epoch 87/100
196/196 [==============================] - 1s 3ms/step - loss: 3.9324e-04 - accuracy: 1.0000
Epoch 88/100
196/196 [==============================] - 1s 3ms/step - loss: 3.5756e-04 - accuracy: 1.0000
Epoch 89/100
196/196 [==============================] - 1s 3ms/step - loss: 3.2757e-04 - accuracy: 1.0000
Epoch 90/100
196/196 [==============================] - 1s 

HM based optimizer model

In [15]:
model_hm = get_model()
model_hm.set_weights(weights) 
st = time.time()
model_hm.fit(X_train_normalized, y_train, epochs = num_epochs, verbose=1, callbacks=[CustomCallbackHM(),logger_hm_model], batch_size=batch_size) 
et = time.time()
elapsed_training_time_hm = round(et - st, 4)
print('Execution time:', elapsed_training_time_hm, 'seconds')

2022-10-15 18:27:55.997968: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 585.94MiB (rounded to 614400000)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-10-15 18:27:55.998053: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2022-10-15 18:27:55.998079: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 59, Chunks in use: 58. 14.8KiB allocated for chunks. 14.5KiB in use in bin. 3.1KiB client-requested in use in bin.
2022-10-15 18:27:55.998093: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 7, Chunks in use: 7. 3.5KiB allocated for chunks. 3.5KiB in use in bin. 3.5KiB client-requested in use in bin.
2022-10-15 18:27:55.998105: I 

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

Model summary

In [ ]:
model_hm.summary()

#### Generic optimizer vs HM-based optimizer

In [ ]:
df1 = pd.read_csv(generic_file)
df2 = pd.read_csv(hm_file)

Accuracy

In [ ]:
x1 = range(0, df1.shape[0])
x2 = range(0, df2.shape[0])
y1 = df1['accuracy'] 
y2 = df2['accuracy']  
plt.figure(figsize = (3,2), dpi = 200)
plt.plot(x1, y1, "r-", label = opti_name, linewidth = 0.8, alpha = 0.7)
plt.plot(x2, y2, "k:", label = 'HM-based ' + opti_name, linewidth = 1, alpha = 0.9) 
plt.ylabel('Accuracy' , fontdict = {'fontsize':5})
plt.xlabel('Epoch', fontdict = {'fontsize':5}) 
#plt.title("Loss", fontdict = {'fontname':'Times New Roman', 'fontsize':8})
plt.xticks(fontsize = 5)
plt.yticks(fontsize = 5)
plt.tight_layout()
plt.legend(prop={'size': 5})
#plt.savefig("graph.png",bbox_inches='tight',dpi=(300)) 
plt.show() 

Loss

In [ ]:
x1 = range(0, df1.shape[0])
x2 = range(0, df2.shape[0])
y1 = df1['loss'] 
y2 = df2['loss']   
plt.figure(figsize = (3,2), dpi = 200)
plt.plot(x1, y1, "r-", label = opti_name, linewidth = 0.8, alpha = 0.7)
plt.plot(x2, y2, "k:", label = 'HM-based ' + opti_name, linewidth = 1, alpha = 0.9) 
plt.ylabel('Loss' , fontdict = {'fontsize':5})
plt.xlabel('Epoch', fontdict = {'fontsize':5}) 
#plt.title("Loss", fontdict = {'fontname':'Times New Roman', 'fontsize':8})
plt.xticks(fontsize = 5)
plt.yticks(fontsize = 5)
plt.tight_layout()
plt.legend(prop={'size': 5})
#plt.savefig("graph.png",bbox_inches='tight',dpi=(300)) 
plt.show()  

###### Testing the model

Generic opimizer model

In [ ]:
model_wihtout_hm.evaluate(X_test_normalized, y_test)

HM based optimizer model

In [ ]:
model_hm.evaluate(X_test_normalized, y_test)

#### Release the GPU memory